In [ ]:
!pip -q install ../input/timm-0-1-30/timm-0.1.30-py3-none-any.whl

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import cv2
import timm
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.models as models
import pytorch_lightning as pl
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
%matplotlib inline
from pylab import rcParams

In [ ]:
df_train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
data_dir = '../input/cassava-leaf-disease-classification/train_images/'

In [ ]:
DEBUG = True  # set it False for full training

In [ ]:
class config:
    FOLD_ID = 0
    IMAGE_SIZE = 256
    BATCH_SIZE = 32
    EPOCHS = 10
    LR = 1e-3
    NWORKERS = 24
    SEED = 42
    NSPLITS = 5
    NCLASSES = 5
    T_max = 10
cfg = config

In [ ]:
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
if DEBUG:
    df_train = df_train[:200]
    
skf = StratifiedKFold(cfg.NSPLITS, random_state=cfg.SEED, shuffle=True)
df_train['fold'] = -1
for fold, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train.label)):
    df_train.loc[valid_idx, 'fold'] = fold

In [ ]:
df_train.head()

In [ ]:
transforms_train = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.Transpose(),
    
    A.OneOf([
            A.CLAHE(clip_limit=2),
            A.IAASharpen(),
            A.IAAEmboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.3),
    
    A.Resize(cfg.IMAGE_SIZE, cfg.IMAGE_SIZE),
    A.Normalize(),
    ToTensorV2()
])


transforms_valid = A.Compose([
    A.Resize(cfg.IMAGE_SIZE, cfg.IMAGE_SIZE),
    A.Normalize(),
    ToTensorV2()
])

In [ ]:
class CLDDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        label = row.label
        image = cv2.imread(data_dir + row.image_id)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            auged = self.transform(image=image)
            image = auged['image']
        
        return image, label

In [ ]:
dataset = CLDDataset(df_train, transforms_train)

In [ ]:
rcParams['figure.figsize'] = 20,10
for i in range(2):
    fig, ax = plt.subplots(1,5)
    for p in range(5):
        img, label = dataset[i*5+p]
        ax[p].imshow(img.permute(1,2,0))

In [ ]:
model = timm.create_model('tf_efficientnet_b0_ns', pretrained=True, num_classes=cfg.NCLASSES)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
class CLDC(LightningModule):
    def __init__(self, df, model, criterion):
        super().__init__()
        self.df = df
        self.net = model
        self.criterion = criterion


    def forward(self, x):
        output = self.net(x)
        return output

    def training_step(self, batch, batch_idx):
        imgs, labels = batch

        x = self(imgs)
        loss = self.criterion(x, labels)
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        x = self(imgs)
        val_loss = self.criterion(x, labels)
        
        self.log('val_loss', val_loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return val_loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=cfg.LR)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.T_max)
        return [optimizer], [scheduler]

    def train_dataloader(self):
        train_set = CLDDataset(df=self.df[self.df['fold'] != cfg.FOLD_ID], transform=transforms_train)
        train_loader = DataLoader(train_set, batch_size=cfg.BATCH_SIZE, drop_last=True)

        return train_loader

    def val_dataloader(self):
        val_set = CLDDataset(df=self.df[self.df['fold'] == cfg.FOLD_ID], transform=transforms_valid)
        val_loader = DataLoader(val_set, batch_size=cfg.BATCH_SIZE)

        return val_loader

In [ ]:
cldc = CLDC(df=df_train, model=model, criterion=criterion)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
    save_weights_only=False
)

In [ ]:
trainer = pl.Trainer(
    gpus=-1,
    max_epochs=cfg.EPOCHS,
    benchmark=True,
    amp_level='O1',
    # auto_lr_find=True,
    checkpoint_callback=checkpoint_callback
)
trainer.fit(cldc)